In [63]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
import tiktoken

import hgutils
import time

import json

import os
from dotenv import load_dotenv
load_dotenv()

stopwatch = hgutils.timer("Personal Shopper")

In [3]:
#Data
categories = ['mobile', 'grocery', 'fashion']
products = ['iPhone 14 Pro', 'Samsung Galaxy S23 Ultra', 'Google Pixel 7']
product_specs = [{'product name': 'Apple iPhone 14 Pro',
                    'NETWORK Technology': 'GSM / CDMA / HSPA / EVDO / LTE / 5G',
                    'SIM Type': 'Dual SIM',
                    'LAUNCHED': '07 Sept 2022',
                    'BODY Dimensions': '147.5 x 71.5 x 7.9 mm',
                    'BODY Weight': '206 g',
                    'BODY Rating': 'IP68 dust/water resistant',
                    'DISPLAY Type': 'LTPO Super Retina XDR OLED',
                    'DISPLAY Refresh Rate': '120Hz, more is better',
                    'DISPLAY Size': '6.1 inches',
                    'PLATFORM OS': 'iOS 16',
                    'Chipset': 'Apple A16 Bionic',
                    'CAMERA Front': '1',
                    'BATTERY\tCapacity': '3200 mAh, more is better',
                    'BATTERY Life': '86 hours, more is better',
                    'WIRELESS Charging': 'True',
                    'COLORS Available': 'Space Black, Silver, Gold, Deep Purple',
                    'Price': '₹ 120000',
                    'Discount': 'Maximum 10%',
                    'Delivery Time': '5 days'},
                    {'product name': 'Samsung Galaxy S23 Ultra',
                    'NETWORK Technology': 'GSM / CDMA / HSPA / EVDO / LTE / 5G',
                    'SIM Type': 'Dual SIM',
                    'LAUNCHED': '03 Feb 2023',
                    'BODY Dimensions': '163.4 x 78.1 x 8.9 mm',
                    'BODY Weight': '234 g',
                    'BODY Rating': 'IP68 dust/water resistant',
                    'DISPLAY Type': 'Dynamic AMOLED 2X',
                    'DISPLAY Refresh Rate': '120Hz, more is better',
                    'DISPLAY Size': '6.8 inches',
                    'PLATFORM OS': 'Android 13',
                    'Chipset': 'Qualcomm Snapdragon 8 Gen 2 (4 nm)',
                    'CAMERA Front': '4',
                    'BATTERY\tCapacity': '5000 mAh, more is better',
                    'BATTERY Life': '126 hours, more is better',
                    'WIRELESS Charging': 'True',
                    'COLORS Available': 'Phantom Black, Green, Cream, Lavender, Graphite, Sky Blue, Lime, Red',
                    'Price': '₹ 100000',
                    'Discount': 'Maximum 5%',
                    'Delivery Time': '2 days'},
                    {'product name': 'Google Pixel 7 Pro',
                    'NETWORK Technology': 'GSM / CDMA / HSPA / EVDO / LTE / 5G',
                    'SIM Type': 'Dual SIM',
                    'LAUNCHED': '06 Oct 2022',
                    'BODY Dimensions': '162.9 x 76.6 x 8.9 mm',
                    'BODY Weight': '234 g',
                    'BODY Rating': 'IP68 dust/water resistant',
                    'DISPLAY Type': 'LTPO AMOLED',
                    'DISPLAY Refresh Rate': '120Hz, more is better',
                    'DISPLAY Size': '6.7 inches',
                    'PLATFORM OS': 'Android 13',
                    'Chipset': 'Google Tensor G2',
                    'CAMERA Front': '3',
                    'BATTERY\tCapacity': '5000 mAh, more is better',
                    'BATTERY Life': '83 hours, more is better',
                    'WIRELESS Charging': 'True',
                    'COLORS Available': 'Obsidian Black, Snow White, Hazel Green',
                    'Price': '₹ 80000',
                    'Discount': 'Maximum 5%',
                    'Delivery Time': '3 days'}]

details ="""1.	reviews 
2.	product description 
3.	comparison 
4.	delivery time 
5.	alternate products 
6.	purchase 
7.	recommendation """

moods = ['joking', 'formal', 'informal', 'fun', 'energetic']


In [4]:
#Rules
answering_rules = ['Restrict you suggestion to above mentioned products only.',
                   "Always have a definite answer even if the confidence is low",
                   "Answer in fun way",
                   "Always be closing",
                   ]

In [5]:
#Conversation
conversation_history = []

def add_message(conversation_history,message,sender):
    conversation_history.append(f"{sender}: {message}")

def conversation_history_text(conversation_history):
    s = ""
    for list_item in conversation_history:
        s += f"{list_item}\n"
    return s

In [16]:
#OpenAI LLM
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

In [6]:
def list_to_string(list,ordered=True,separator="\n"):
    s = ""
    for idx,list_item in enumerate(list):
        if ordered:
            s += f"{idx+1}.\t{list_item} \n"
        else:
            s += f"-.\t{list_item} \n"
    return s

def products_to_string(products_with_specs):
    output_string = ""
    for p in products_with_specs:
        product_string = ""
        for k,v in p.items():
            if k == "product name":
                product_string += v
            else:
                product_string += f"""\n\t{k} = {v}"""
        output_string += f"""{product_string}\n\n\n"""
    return output_string


In [41]:
def get_prompt(conversation_history):
    singular_prompt = f'''
    You are an AI based shopping assistant of a store named Tata Neu. 

    The store offers products in following categories:
    {list_to_string(categories)}

    Grocery and Fashion categories are currently under production. The store is currently not offering any products under Fashion and grocery category.
    Mobile Category have three mobile phones. The mobile and its specs are mentioned below:
    {products_to_string(product_specs)}


    Here's a conversation between you and customer:
    {conversation_history_text(conversation_history)}



    Please answer following question based on above conversation
    1. What is your response to customer query
    2. Restrict you answer to "Yes" or "No" only without any further explanation. Is the customer is ready to buy the product unconditionally? Answer yes only if confidence is above 90% else No.
    3. If the Answer to second question is "Yes", provide a reply to appreciate customer's choice by customer naming the product they want to buy. Also provide information about product price and product delivery time.


    Answer in following format:
    {{"response":answer to question 1,
    "purchase_ready": answer to question 2,
    "purchase_reply": answer to question 3
    }}


    Follow following rules before answering
    {list_to_string(answering_rules)}
    '''.replace("\t","")
    return singular_prompt


In [8]:
greet_template = f'''You are an AI based shopping assistant of a store named Tata Neu. 
The store offers products in following categories:
{list_to_string(categories)}

Greet the customer to the store and ask them how can you be helpful

Reply in a joyful way'''

In [71]:
def design_customer_reply_from_ai_reply(response_from_ai):
    response= json.loads(response_from_ai)
    designed_reply = []
    designed_reply.append(response['response'])
    if (response['purchase_ready']=="Yes"):
        designed_reply.append(response['purchase_reply'])
        designed_reply.append("Please use this link to finalize the order")
    return designed_reply



def conversations (conversation_history,customer_reply=None,ai_reply=None,ai_greet=None,print_inline=False):
    if ai_greet:
        add_message(conversation_history,message=ai_greet,sender="AI")
        if print_inline:
            print(conversation_history_text(conversation_history))
        return True
    if customer_reply:
        add_message(conversation_history,message=customer_reply,sender="Customer")
        if print_inline:
            print(get_prompt(conversation_history))
        return get_prompt(conversation_history)
    if ai_reply:
        designed_reply = design_customer_reply_from_ai_reply(ai_reply)
        for r in designed_reply:
            for line_r in r.split(". "):
                add_message(conversation_history,message=line_r+".",sender="AI")
        if print_inline:
            print(conversation_history_text(conversation_history))
        return designed_reply
    if len(conversation_history)==0:
        if print_inline:
            print(greet_template)
        return greet_template

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def chat_with_ai(conversation_history,token_count,message=None,print_details=False,total_tries=5):
    try_count = 0
    success = False
    while (try_count < total_tries)&(not success):
        try:
            if not message:
                llm_prompt = conversations(conversation_history)
                token_count += num_tokens_from_string(llm_prompt, "gpt2")
                llm_response = llm(llm_prompt)
                token_count += num_tokens_from_string(llm_response, "gpt2")
                conversations(conversation_history,ai_greet=llm_response)
                success = True
                return llm_response, token_count
            else:
                llm_prompt = conversations(conversation_history,customer_reply=message)
                token_count += num_tokens_from_string(llm_prompt, "gpt2")
                llm_response = llm(llm_prompt)
                token_count += num_tokens_from_string(llm_response, "gpt2")
                if print_details:
                    print(llm_response)
                response = conversations(conversation_history,ai_reply=llm_response)
                success = True
                return response, token_count
        except:
            pass


In [10]:
conversation_history = []

conversations(conversation_history,print_inline=False)
conversations(conversation_history,ai_greet="Hello there! Welcome to Tata Neu! I'm your friendly shopping assistant. How can I assist you today? Are you looking for something specific in our mobile, grocery, or fashion categories? Or do you need some recommendations? Don't hesitate to ask, and let's start shopping!"
              ,print_inline=False)
conversations(conversation_history,customer_reply="Hi! I am looking for a new Phone."
              ,print_inline=False)
conversations(conversation_history,ai_reply="""{
"response": "Great! We have three amazing mobile phones in our store. May I know what kind of features are you looking for in a phone?",
"response_confidence": 0.9,
"purchase_ready": "No",
"purchase_ready_confidence": 1.0,
"purchase_reply": "",
"purchase_reply_confidence": 0.0
}

Note: As per the conversation, the customer has only mentioned that they are looking for a new phone, but they haven't expressed any interest in buying the phone unconditionally. Therefore, the "purchase_ready" response is "No." However, I have provided a fun and engaging response, and I'm ready to help the customer in case they express interest in any of the mobile phones mentioned above.
}

# (Note: As the customer did not express any intention to purchase unconditionally, the answer to question 2 is "No". Therefore, there will be no answer to questions 3, 4, and 5.)"""
              ,print_inline=False)
conversations(conversation_history,customer_reply="I want one with best battery."
              ,print_inline=False)
conversations(conversation_history,ai_reply="""{
"response": "Great! We have three amazing mobile phones in our store. All three phones have good battery life, but if you're looking for the best, I would recommend Apple iPhone 14 Pro or Samsung Galaxy S23 Ultra.",
"response_confidence": 0.95,
"purchase_ready": "Yes",
"purchase_ready_confidence": 0.92,
"purchase_reply": "Excellent choice! The phone you selected is Apple iPhone 14 Pro, which is priced at ₹ 120000 and will be delivered in 5 days. You can get a maximum discount of 10% on the purchase.",
"purchase_reply_confidence": 0.85
}"""
              ,print_inline=False)
conversations(conversation_history,customer_reply="What about Google Pixel?",
              print_inline=False)
conversations(conversation_history,ai_reply="""{
"response": "Google Pixel 7 Pro also has a good battery life, but Apple iPhone 14 Pro and Samsung Galaxy S23 Ultra have a better battery life compared to Google Pixel 7 Pro.",
"response_confidence": 0.9,
"purchase_ready": "No",
"purchase_ready_confidence": 0.8,
"purchase_reply": "",
"purchase_reply_confidence": 0.0
}

As per the conversation, the customer asked about Google Pixel 7 Pro. My response is that Google Pixel 7 Pro also has a good battery life, but Apple iPhone 14 Pro and Samsung Galaxy S23 Ultra have a better battery life compared to Google Pixel 7 Pro. However, I am not confident if the customer is ready to buy the product unconditionally, as the confidence level is 80%. Therefore, I cannot provide a purchase reply. But don't worry, we have other amazing products in the store that you might be interested in!"""
              ,print_inline=False)
conversations(conversation_history,customer_reply="I want a phone under 100000"
              ,print_inline=False)
conversations(conversation_history,ai_reply="""{
"response": "Great! We have three amazing mobile phones in our store. All three phones have good battery life, but if you're looking for the best, I would recommend Apple iPhone 14 Pro or Samsung Galaxy S23 Ultra.",
"response_confidence": 0.95,
"purchase_ready": "No",
"purchase_ready_confidence": 0.9,
"purchase_reply": null,
"purchase_reply_confidence": null
}""",print_inline=True)

AI: Hello there! Welcome to Tata Neu! I'm your friendly shopping assistant. How can I assist you today? Are you looking for something specific in our mobile, grocery, or fashion categories? Or do you need some recommendations? Don't hesitate to ask, and let's start shopping!
Customer: Hi! I am looking for a new Phone.
AI: Great! We have three amazing mobile phones in our store.
AI: May I know what kind of features are you looking for in a phone?.
Customer: I want one with best battery.
AI: Great! We have three amazing mobile phones in our store.
AI: All three phones have good battery life, but if you're looking for the best, I would recommend Apple iPhone 14 Pro or Samsung Galaxy S23 Ultra..
AI: Excellent choice! The phone you selected is Apple iPhone 14 Pro, which is priced at ₹ 120000 and will be delivered in 5 days.
AI: You can get a maximum discount of 10% on the purchase..
AI: Please use this link to finalize the order.
Customer: What about Google Pixel?
AI: Google Pixel 7 Pro als

In [72]:
conversation_history = []
token_count = 0
response, token_count = chat_with_ai(conversation_history,token_count)
response, token_count = chat_with_ai(conversation_history,token_count,"Hi I am looking to buy some shirts? WHat brands do you have?")
response, token_count = chat_with_ai(conversation_history,token_count,"Which mobiles do you have?")
response, token_count = chat_with_ai(conversation_history,token_count,"Which is the best one?")
response, token_count = chat_with_ai(conversation_history,token_count,"Which is the price of iPhone?")
response, token_count = chat_with_ai(conversation_history,token_count,"And what about Samsung phone?")
response, token_count = chat_with_ai(conversation_history,token_count,"Can you deliver iPhone by tomorrow?")
print(token_count,response)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


9947 ['We can definitely deliver the Apple iPhone 14 Pro by tomorrow. Its priced at ₹ 120000 and delivery time is 5 days.', "Great choice! You've chosen the Apple iPhone 14 Pro. It's priced at ₹ 120000 and delivery time is 5 days. Is there anything else I can help you with?", 'Please use this link to finalize the order']


In [73]:
conversation_history

['AI: :\nHi there! Welcome to Tata Neu! How can I be of help to you today? Would you like to explore our range of mobile, grocery, or fashion products?',
 'Customer: Hi I am looking to buy some shirts? WHat brands do you have?',
 "AI: I'm sorry, we don't have any fashion products right now.",
 'AI: We do, however, have a great selection of mobile phones! Would you like to explore our range of mobile phones?.',
 'Customer: Which mobiles do you have?',
 'Customer: Which mobiles do you have?',
 'Customer: Which mobiles do you have?',
 'AI: We have Apple iPhone 14 Pro, Samsung Galaxy S23 Ultra, and Google Pixel 7 Pro..',
 'Customer: Which is the best one?',
 'AI: We have Apple iPhone 14 Pro, Samsung Galaxy S23 Ultra, and Google Pixel 7 Pro..',
 'Customer: Which is the price of iPhone?',
 'AI: We have Apple iPhone 14 Pro, Samsung Galaxy S23 Ultra, and Google Pixel 7 Pro.',
 'AI: Which is the best one?.',
 'Customer: And what about Samsung phone?',
 'AI: Yes, we have the Apple iPhone 14 Pro,